# PL Standings

### This is for testing purposes

In [64]:
import os
import json
import requests
import pandas as pd
from mysql import connector
from dotenv import load_dotenv
from datetime import datetime


In [65]:
load_dotenv()

True

In [66]:
API_KEY = os.getenv("API_KEY")
API_HOST = os.getenv("API_HOST")

print(API_KEY)

b41edd5e36msh3f891717feee202p1e253cjsn50e0cde77a5e


In [ ]:
url = "https://odds.p.rapidapi.com/v4/sports/upcoming/odds"
headers = {
	"x-rapidapi-key": API_KEY,
	"x-rapidapi-host": API_HOST
}

querystring = { "regions"    : "us",
                "oddsFormat" : "decimal",
                "markets"    : "h2h,spreads",
                "dateFormat" : "iso"}


In [68]:
print(querystring)

{'regions': 'us', 'oddsFormat': 'decimal', 'markets': 'h2h,spreads', 'dateFormat': 'iso'}


In [69]:
response = requests.get(url = url,
                        headers = headers,
                        params = querystring)

match_list = response.json()

In [70]:
formatted_response = json.dumps(match_list, indent = 4)

print(formatted_response)

[
    {
        "id": "611ebd7ee449d34c85e1dee5c7fe0fbe",
        "sport_key": "cricket_international_t20",
        "sport_title": "International Twenty20",
        "commence_time": "2025-09-29T14:30:00Z",
        "home_team": "Nepal",
        "away_team": "West Indies",
        "bookmakers": [
            {
                "key": "fanduel",
                "title": "FanDuel",
                "last_update": "2025-09-29T16:22:11Z",
                "markets": [
                    {
                        "key": "h2h",
                        "last_update": "2025-09-29T16:22:11Z",
                        "outcomes": [
                            {
                                "name": "Nepal",
                                "price": 1.91
                            },
                            {
                                "name": "West Indies",
                                "price": 1.91
                            }
                        ]
                    }
          

In [71]:
first_home_team_price = match_list[0]["bookmakers"][0]["markets"][0]["outcomes"][1]["price"]

print(first_home_team_price)

1.91


In [72]:
rows = []
column_names = ["live_match_id","sport_title","commence_time","home_team","away_team","home_team_price","away_team_price"]

for match in match_list:
    live_match_id   = match["id"]
    sport_title     = match["sport_title"]
    commence_time   = datetime.strptime(match["commence_time"], "%Y-%m-%dT%H:%M:%SZ")
    home_team       = match["home_team"]
    away_team       = match["away_team"]
    home_team_price = match["bookmakers"][0]["markets"][0]["outcomes"][1]["price"]
    away_team_price = match["bookmakers"][0]["markets"][0]["outcomes"][0]["price"]

    match_info_tuple = (live_match_id, sport_title, commence_time, home_team, away_team, home_team_price, away_team_price)

    rows.append(match_info_tuple)



In [73]:
print(rows)

[('611ebd7ee449d34c85e1dee5c7fe0fbe', 'International Twenty20', datetime.datetime(2025, 9, 29, 14, 30), 'Nepal', 'West Indies', 1.91, 1.91), ('efd9294108d31dcd79fe06462303f3fe', 'Super League - Greece', datetime.datetime(2025, 9, 29, 15, 0), 'Atromitos Athens', 'AEL', 1.91, 8.5), ('4871d5ee2edd5a9c3ddcc01fa420c469', 'Ekstraklasa - Poland', datetime.datetime(2025, 9, 29, 16, 0), 'Zagłębie Lubin', 'Arka Gdynia', 1.07, 20.0), ('5b45a72110489f1c98d2e696445c33c5', 'Veikkausliiga - Finland', datetime.datetime(2025, 9, 29, 16, 0), 'HJK Helsinki', 'Ilves Tampere', 1.74, 3.9), ('a1c1af14f45268b6fd1384b6d627b1df', 'Serie A - Italy', datetime.datetime(2025, 9, 29, 16, 30), 'Parma', 'Torino', 3.1, 2.55), ('71922aa46ec6b8d8e8707fe06656ea72', 'HockeyAllsvenskan', datetime.datetime(2025, 9, 29, 17, 0), 'AIK', 'Nybro Vikings IF', 2.5, 1.5), ('4ddc1a891ca4cc1dfa656b0d83b7628a', 'Turkey Super League', datetime.datetime(2025, 9, 29, 17, 0), 'Besiktas JK', 'Kocaelispor', 7.5, 1.33), ('306bc56e9b1504a8b9f3

In [74]:
df = pd.DataFrame(rows, columns=column_names)

df.head(20)

,live_match_id,sport_title,commence_time,home_team,away_team,home_team_price,away_team_price
0,611ebd7ee449d34c85e1dee5c7fe0fbe,International Twenty20,2025-09-29 14:30:00,Nepal,West Indies,1.91,1.91
1,efd9294108d31dcd79fe06462303f3fe,Super League - Greece,2025-09-29 15:00:00,Atromitos Athens,AEL,1.91,8.50
2,4871d5ee2edd5a9c3ddcc01fa420c469,Ekstraklasa - Poland,2025-09-29 16:00:00,Zagłębie Lubin,Arka Gdynia,1.07,20.00
3,5b45a72110489f1c98d2e696445c33c5,Veikkausliiga - Finland,2025-09-29 16:00:00,HJK Helsinki,Ilves Tampere,1.74,3.90
4,a1c1af14f45268b6fd1384b6d627b1df,Serie A - Italy,2025-09-29 16:30:00,Parma,Torino,3.10,2.55
5,71922aa46ec6b8d8e8707fe06656ea72,HockeyAllsvenskan,2025-09-29 17:00:00,AIK,Nybro Vikings IF,2.50,1.50
6,4ddc1a891ca4cc1dfa656b0d83b7628a,Turkey Super League,2025-09-29 17:00:00,Besiktas JK,Kocaelispor,7.50,1.33
7,306bc56e9b1504a8b9f3119a08e0f617,Allsvenskan - Sweden,2025-09-29 17:00:00,Djurgardens IF,IK Sirius,4.00,1.74
8,dc4d63b69c11b1427e8dc3f374e99beb,Denmark Superliga,2025-09-29 17:00:00,FC Midtjylland,Randers FC,4.75,1.66
9,a4dde9fe8240a6d28d21ee3c14668698,Superettan - Sweden,2025-09-29 17:00:00,GIF Sundsvall,Umeå FC,4.20,1.70


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   live_match_id    12 non-null     object        
 1   sport_title      12 non-null     object        
 2   commence_time    12 non-null     datetime64[ns]
 3   home_team        12 non-null     object        
 4   away_team        12 non-null     object        
 5   home_team_price  12 non-null     float64       
 6   away_team_price  12 non-null     float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 804.0+ bytes


In [76]:
MYSQL_HOST      = os.getenv("MYSQL_HOST")
MYSQL_PORT      = os.getenv("MYSQL_PORT")
MYSQL_USER      = os.getenv("MYSQL_USER")
MYSQL_PASSWORD  = os.getenv("MYSQL_PASSWORD")
MYSQL_DATABASE      = os.getenv("MYSQL_DATABASE")

In [77]:
server_conn = connector.connect(
    host = MYSQL_HOST,
    port = MYSQL_PORT,
    user = MYSQL_USER,
    password = MYSQL_PASSWORD,
    connection_timeout = 10,
    autocommit = False,
    raise_on_warnings = True,
)

server_cur = server_conn.cursor()

print(f"[SUCCESS] - Connected to MySQL Server")

[SUCCESS] - Connected to MySQL Server


In [78]:
server_cur.close()
server_conn.close()

In [79]:
db_connection = connector.connect(
    host = MYSQL_HOST,
    port = MYSQL_PORT,
    user = MYSQL_USER,
    password = MYSQL_PASSWORD,
    database = MYSQL_DATABASE
)

cur = db_connection.cursor()

print(f"[SUCCESS] - Connection to database is successful!")

[SUCCESS] - Connection to database is successful!


In [80]:
sql_table = "matches"
cur.execute("SHOW TABLES LIKE %s",(f"{sql_table}",))

if cur.fetchone() is None:
    raise SystemExit(f'Table "{sql_table}" is NOT found, please create it')

else:
    print(f'[SUCCESS] This table "{sql_table}" exists! Continue to the next phase')

[SUCCESS] This table "matches" exists! Continue to the next phase


In [81]:
# Start the UPSERT operation

table_cols = ["live_match_id","sport_title","commence_time","home_team","away_team","home_team_price","away_team_price"]

matches_df = df[table_cols]



In [82]:
matches_records_tuples = matches_df.itertuples(index = False, name = None)

list_of_matches_records_tuples = list(matches_records_tuples)

In [83]:
UPSERT_SQL = f"""
INSERT INTO {sql_table}
(live_match_id, sport_title, commence_time, home_team, away_team, home_team_price, away_team_price)
VALUES (%s, %s, %s, %s, %s, %s, %s) AS src
ON DUPLICATE KEY UPDATE
live_match_id       = src.live_match_id,
sport_title         = src.sport_title,
commence_time       = src.commence_time,
home_team           = src.home_team,
away_team           = src.away_team,
home_team_price     = src.home_team_price,
away_team_price     = src.away_team_price;
"""

In [84]:
no_of_rows_uploaded_to_mysql = len(list_of_matches_records_tuples)

In [85]:
try:
    cur.executemany(UPSERT_SQL, list_of_matches_records_tuples)
    db_connection.commit()
    print(f"[SUCCESS] - Upsert attempted for {no_of_rows_uploaded_to_mysql} rows!")

except Exception as e:
    db_connection.rollback()
    print(f"[ERROR] - Rolled back due to this: {e}")

finally:
    cur.close()
    db_connection.close()
    print("All database connections now closed. \n\nClean up completed.")

[SUCCESS] - Upsert attempted for 12 rows!
All database connections now closed. 

Clean up completed.
